In [1]:
import sys
sys.path.append('../')

import numpy as np
from scipy import ndimage

from skimage.filters import sobel_h
from skimage.filters import sobel_v
from scipy import stats


import os
import matplotlib
import matplotlib.pyplot as plt
import scienceplots
from tensorflow.python.client import device_lib

#plt.rcParams['figure.figsize'] = [10,10]

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications import VGG16

from tensorflow.nn import depthwise_conv2d
from tensorflow.math import multiply, reduce_sum, reduce_mean,reduce_euclidean_norm, sin, cos, abs, logical_and
from tensorflow import stack, concat, expand_dims


import tensorflow_probability as tfp

from utils.utils import *


plt.style.use(['science', 'ieee'])
plt.rcParams.update({'figure.dpi': '100'})

#### Experiment #1 : $\beta$ accross all layers (Top-10% filters)

In [2]:
model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

In [3]:
k = 20 # Top 10% highest l2-magnitude filters

In [5]:
conv_layers = []
for l in model.layers:
    if 'conv2d' in str(type(l)).lower():
        conv_layers.append(l)

a_count= []
s_count= []
m_count = []

full_a_a = []
full_a_s = []
full_a_m = []

full_s_a = []
full_s_s = []
full_s_m = []

full_m_a = []
full_m_s = []
full_m_m = []

for i, l in enumerate(conv_layers[1:]):
    l_num = i+1
    print(f" ============ LAYER {l_num} , {l.name}==================")
    top10F=topKfilters(model,l_num, k=k)
    print(l_num, top10F)
    filters = get_filter(model, l_num)[:,:,:, top10F]
    s, a = getSymAntiSymTF(filters)
    a_e = reduce_euclidean_norm(a, axis=[0,1])**2
    #print(a_mag.shape)

    s_e = reduce_euclidean_norm(s, axis=[0,1])**2
    beta = tf.reduce_mean(a_e/(s_e+a_e), axis=0)


    antisym_filters = tf.reshape(tf.where(beta >= 0.75), [-1])
    sym_filters = tf.reshape(tf.where(beta <= 0.25), [-1])
    mix_filters = tf.reshape(tf.where(logical_and(beta > 0.25, beta<0.75)), [-1])

    a_count.append(len(antisym_filters))
    s_count.append(len(sym_filters))
    m_count.append(len(mix_filters))
    #print(antisym_filters)

    '''a_mag = reduce_euclidean_norm(a, axis=[0,1, 2])**2
    a_mag = a_mag[top10F]
    s_mag = reduce_euclidean_norm(s, axis=[0,1, 2])**2
    s_mag = s_mag[top10F]'''

    a_a = 0
    a_s = 0
    s_a = 0
    s_s = 0
    m_a = 0
    m_s = 0
    m_m = 0
    a_m = 0
    s_m = 0

    prv_top10KF = topKfilters(model, l_num-1, k=k)
    prv_filters = get_filter(model, l_num-1)[:,:,:, prv_top10KF]
    prv_s, prv_a = getSymAntiSymTF(prv_filters)
    prv_a_mag = reduce_euclidean_norm(prv_a, axis=[0,1])**2
    prv_s_mag = reduce_euclidean_norm(prv_s, axis=[0,1])**2
    prv_avg_amag = reduce_mean(prv_a_mag, axis=0).numpy()
    prv_avg_smag = reduce_mean(prv_s_mag, axis=0).numpy()
    prv_beta = reduce_mean(prv_a_mag/(prv_a_mag+prv_s_mag), axis=0).numpy()
    prv_antisym_filters = list(np.array(topKfilters(model, l_num-1, k=100))[[tf.reshape(tf.where(prv_beta >= 0.75), (-1))]])
    prv_sym_filters = list(np.array(topKfilters(model, l_num-1, k=100))[[tf.reshape(tf.where(prv_beta <= 0.25), (-1))]])
    prv_mixed_filters = list(np.array(topKfilters(model, l_num-1, k=100))[[tf.reshape(tf.where(logical_and(prv_beta > 0.25, prv_beta<0.75)), (-1))]])

    for i, idx in enumerate(top10F) :
        #print("Filter ", idx , " : ", topKchannels(l_num+1,idx),"--->",set(topKchannels(l_num+1,idx)).intersection(top10F))
        #Is this filter symetric or antisymmetric ?
        is_anti = beta[i] >= 0.75
        is_mixed = beta[i] <0.75 and beta[i]>0.25
        print(is_anti, is_mixed)
        #print()
        # Out of its top-k connections, how many inputs are comming from sym-filters, 
        #                               how many inputs are comming from antisym-filters?

        top_k_connections = list(set(topKchannels(model,l_num,idx, k=k)).intersection(prv_top10KF))
        print("TKC",top_k_connections)
        #Check : does top_k_connections lie in antisym_filters or in sym_filters (or neither) ?  
        #print (top_k_connections, antisym_filters.shape)
        #print(tf.squeeze(antisym_filters) )

        #print("HERE" , set(tf.squeeze(antisym_filters).numpy()).intersection(set(top_k_connections)))
        #print("HERE" , set(tf.squeeze(sym_filters).numpy()).intersection(set(top_k_connections)))
        if is_anti: 
            a_a += len(list(set(prv_antisym_filters).intersection(set(top_k_connections))))
            print(idx, "Anti -> Anti : ", list(set(prv_antisym_filters).intersection(set(top_k_connections))))
            s_a += len(list(set(prv_sym_filters).intersection(set(top_k_connections))))
            print(idx,"Sym -> Anti : ", list(set(prv_sym_filters).intersection(set(top_k_connections))))
            m_a += len(list(set(prv_mixed_filters).intersection(set(top_k_connections))))
            print(idx,"Mixed -> Anti : ", list(set(prv_mixed_filters).intersection(set(top_k_connections))))
        elif is_mixed:
            a_m += len(list(set(prv_antisym_filters).intersection(set(top_k_connections))))
            print(idx, "Anti -> Mixed : ", list(set(prv_antisym_filters).intersection(set(top_k_connections))))
            s_m += len(list(set(prv_sym_filters).intersection(set(top_k_connections))))
            print(idx,"Sym -> Mixed : ", list(set(prv_sym_filters).intersection(set(top_k_connections))))
            m_m += len(list(set(prv_mixed_filters).intersection(set(top_k_connections))))
            print(idx,"Mixed -> Mixed : ", list(set(prv_mixed_filters).intersection(set(top_k_connections))))
        else:
            a_s += len(list(set(prv_antisym_filters).intersection(set(top_k_connections))))
            print(idx, "Anti -> Sym : ", list(set(prv_antisym_filters).intersection(set(top_k_connections))))
            s_s += len(list(set(prv_sym_filters).intersection(set(top_k_connections))))
            print(idx,"Sym -> Sym : ", list(set(prv_sym_filters).intersection(set(top_k_connections))))
            m_s += len(list(set(prv_mixed_filters).intersection(set(top_k_connections))))
            print(idx,"Mixed -> Sym : ", list(set(prv_mixed_filters).intersection(set(top_k_connections))))

    num_connections = a_a+a_s+a_m + s_a+s_s+s_m + m_a+m_m+m_s
    full_a_a.append(a_a/num_connections)
    full_a_s.append(a_s/num_connections)
    full_a_m.append(a_m/num_connections)

    full_s_a.append(s_a/num_connections)
    full_s_s.append(s_s/num_connections)
    full_s_m.append(s_m/num_connections)

    full_m_a.append(m_a/num_connections)
    full_m_s.append(m_s/num_connections)
    full_m_m.append(m_m/num_connections)

    print("A -> A ",  a_a)
    print("A -> S ",  a_s)
    print("A -> M ",  a_m)
    print("S -> A ",  s_a)
    print("S -> S ",  s_s)
    print("S -> M ",  s_m)
    print("M -> M ",  m_m)
    print("M -> A ",  m_a)
    print("M -> S ",  m_s)
    print(top_k_connections)


top10F=topKfilters(model,-1, k=k)
print(l_num, top10F)
filters = get_filter(model, l_num)[:,:,:, top10F]
s, a = getSymAntiSymTF(filters)
a_e = reduce_euclidean_norm(a, axis=[0,1])**2
#print(a_mag.shape)

s_e = reduce_euclidean_norm(s, axis=[0,1])**2
beta = tf.reduce_mean(a_e/(s_e+a_e), axis=0)


antisym_filters = tf.reshape(tf.where(beta >= 0.5), [-1])
sym_filters = tf.reshape(tf.where(beta < 0.5), [-1])

a_count.append(len(antisym_filters))
s_count.append(len(sym_filters))
m_count.append(len(mix_filters))


 ============ LAYER 1 , block1_conv2==================
1 [18, 59, 19, 0, 29, 10, 30, 47, 52, 55, 13, 20]


c:\Users\AN46710\Anaconda3\envs\masters\lib\site-packages\ipykernel_launcher.py:68: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
c:\Users\AN46710\Anaconda3\envs\masters\lib\site-packages\ipykernel_launcher.py:69: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
c:\Users\AN46710\Anaconda3\envs\masters\lib\site-packages\ipykernel_launcher.py:70: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(

tf.Tensor(False, shape=(), dtype=bool) tf.Tensor(False, shape=(), dtype=bool)
TKC []
18 Anti -> Sym :  []
18 Sym -> Sym :  []
18 Mixed -> Sym :  []
tf.Tensor(False, shape=(), dtype=bool) tf.Tensor(True, shape=(), dtype=bool)
TKC [34, 3, 37, 41, 45, 51, 58, 62]
59 Anti -> Mixed :  [37, 45, 51, 58, 62]
59 Sym -> Mixed :  []
59 Mixed -> Mixed :  [41, 34, 3]
tf.Tensor(False, shape=(), dtype=bool) tf.Tensor(True, shape=(), dtype=bool)
TKC [0, 34, 3, 37, 41, 10, 45, 62]
19 Anti -> Mixed :  [0, 45, 37, 62]
19 Sym -> Mixed :  []
19 Mixed -> Mixed :  [41, 10, 3, 34]
tf.Tensor(False, shape=(), dtype=bool) tf.Tensor(True, shape=(), dtype=bool)
TKC [0, 3, 37, 10, 47, 51, 58]
0 Anti -> Mixed :  [0, 37, 47, 51, 58]
0 Sym -> Mixed :  []
0 Mixed -> Mixed :  [10, 3]
tf.Tensor(False, shape=(), dtype=bool) tf.Tensor(True, shape=(), dtype=bool)
TKC [0, 37, 10, 45, 51, 58, 62]
29 Anti -> Mixed :  [0, 37, 45, 51, 58, 62]
29 Sym -> Mixed :  []
29 Mixed -> Mixed :  [10]
tf.Tensor(False, shape=(), dtype=bool) 

In [ ]:
s_count

[0, 3, 1, 6, 28, 15, 11, 39, 29, 22, 45, 84, 85]

In [10]:
full_a_a
f = open("figures/vgg16_as_tree_with_mix.tex", "w+")

f.write("\\begin{tikzpicture}\n\\begin{scope}[every node/.style={circle,thick,draw, align=center}, scale=0.8, transform shape]\n ")

pos = len(conv_layers)
for i in range(len(conv_layers)):
    pos -= 3
    f.write(f"    \\node (A_{i}) at (-4,{pos}) {{A\\\\[-0.7em]{a_count[i]}}};\n")
    f.write(f"    \\node (S_{i}) at (0,{pos}) {{S\\\\[-0.7em]{s_count[i]}}};\n")
    f.write(f"    \\node (M_{i}) at (4,{pos}) {{M\\\\[-0.7em]{m_count[i]}}};\n")

    f.write(f"    \\node[align=left, draw=none] at (-4,{pos}) {{{i+1}}};\n")


f.write("\end{scope}")

f.write("\\begin{scope}[>={Stealth[black]},every node/.style={fill=white,circle},every edge/.style={draw=black, thin} ,scale=0.8, transform shape]\n ")
for i in range(len(conv_layers)-1):

    n  = np.argmax([full_a_a[i], full_a_s[i], full_a_m[i], 
                    full_s_a[i], full_s_s[i], full_s_m[i], 
                    full_m_a[i], full_m_s[i] ,full_m_m[i]])
    
    aa_max = "[red, very thick]" if n == 0 else ""
    as_max = "[red, very thick]" if n == 1 else ""
    am_max = "[red, very thick]" if n == 2 else ""

    sa_max = "[red, very thick]" if n == 3 else ""
    ss_max = "[red, very thick]" if n == 4 else ""
    sm_max = "[red, very thick]" if n == 5 else ""

    ma_max = "[red, very thick]" if n == 6 else ""
    ms_max = "[red, very thick]" if n == 7 else ""
    mm_max = "[red, very thick]" if n == 8 else ""

    pos -= 2
    f.write(f"        \\path [->] (A_{i}) edge {as_max}  node[near start]  {{${full_a_s[i]:.2f}$}} (S_{i+1});\n")
    f.write(f"        \\path [->] (A_{i}) edge {aa_max}  node[left] {{${full_a_a[i]:.2f}$}} (A_{i+1});\n")
    f.write(f"        \\path [->] (A_{i}) edge {am_max}  node[left] {{${full_a_m[i]:.2f}$}} (M_{i+1});\n")

    f.write(f"        \\path [->] (S_{i}) edge {sa_max}  node[near start] {{${full_s_a[i]:.2f}$}} (A_{i+1});\n")
    f.write(f"        \\path [->] (S_{i}) edge {ss_max}  node[right] {{${full_s_s[i]:.2f}$}} (S_{i+1});\n")
    f.write(f"        \\path [->] (S_{i}) edge {sm_max}  node[right] {{${full_s_m[i]:.2f}$}} (M_{i+1});\n")

    f.write(f"        \\path [->] (M_{i}) edge {ma_max}  node[near start] {{${full_m_a[i]:.2f}$}} (A_{i+1});\n")
    f.write(f"        \\path [->] (M_{i}) edge {ms_max}  node[right] {{${full_m_s[i]:.2f}$}} (S_{i+1});\n")
    f.write(f"        \\path [->] (M_{i}) edge {mm_max}  node[right] {{${full_m_m[i]:.2f}$}} (M_{i+1});\n")

f.write("\end{scope}")


f.write("\end{tikzpicture}")


f.close()


In [ ]:
full_a_a

[0.9565217391304348,
 0.0,
 0.7222222222222222,
 0.32061068702290074,
 0.319047619047619,
 0.35664335664335667,
 0.42955326460481097,
 0.39285714285714285,
 0.33640552995391704,
 0.3782771535580524,
 0.05098684210526316,
 0.0]